In [1]:
"""
Author:
Date: 25/12/24
"""

from dataclasses import dataclass
import random
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
from typing import List, Callable, Dict, Union, Optional
from datasets import load_dataset
from transformers import (
    Trainer,
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from trl.trainer import grpo_trainer
from grpo_reward_func import correctness_reward, digit_reward, hard_format_reward, mark_reward


/Users/yingyao/miniconda3/envs/transformer-practice/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GSM8KDataset(Dataset):
    def __init__(self, data_path, tokenizer):

        self.tokenizer = tokenizer
        data = load_dataset(data_path)
        self.data = data["train"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        # prompt = self.tokenizer.apply_chat_template(sample['prompt'], tokenize=False, add_generation_prompt=True)
        answer = sample["answer_only"]
        prompt = sample["question"]
        return {"prompt": prompt, "answer": answer}


In [3]:
args = TrainingArguments()
writer = SummaryWriter("./grpo/result/runs")
tokenizer = AutoTokenizer.from_pretrained("Qwen2.5-0.5B-Instruct") # Users/yingyao/Desktop/Code/GetHandsDirty.nosync
policy_model = AutoModelForCausalLM.from_pretrained("Qwen2.5-0.5B-Instruct")

The tokenizer you are loading from 'Qwen2.5-0.5B-Instruct' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


In [4]:
dataset = GSM8KDataset('./dataset/gsm8k_chinese', tokenizer)

In [5]:
dataset[0]

{'prompt': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 72}

In [6]:
@dataclass
class Samples:
    prompt_response_ids: torch.Tensor
    response_ids: torch.Tensor
    prompt: str
    answer: str
    attention_mask: Optional[torch.LongTensor]
    action_mask: Optional[torch.BoolTensor]
    num_actions: Union[torch.Tensor, int]
    response_length: torch.Tensor


@dataclass
class Experience:
    prompt_response_ids: torch.Tensor
    action_log_probs: torch.Tensor
    advantages: Optional[torch.Tensor]
    attention_mask: Optional[torch.LongTensor]
    action_mask: Optional[torch.BoolTensor]
    num_actions: Union[int, torch.Tensor]
    # kl: Optional[torch.Tensor] = None


# @dataclass
# class BufferItem:

#     prompt_response_ids: torch.Tensor
#     action_log_probs: torch.Tensor
#     # values: torch.Tensor
#     # returns: torch.Tensor
#     advantages: torch.Tensor
#     attention_mask: torch.Tensor
#     action_mask: torch.Tensor
#     num_actions: Union[int, torch.Tensor]

# def collate_fn(batch):

#     prompt_response_ids = []
#     action_log_probs = []
#     # values = []
#     # returns = []
#     advantages = []
#     attention_mask = []
#     action_mask = []

#     for x in batch:
#         prompt_response_ids.append(x["prompt_response_ids"])
#         action_log_probs.append(x["action_log_probs"])
#         # values.append(x["values"])
#         # returns.append(x["returns"])
#         advantages.append(x["advantages"])
#         attention_mask.append(x["attention_mask"])
#         action_mask.append(x["action_mask"])

#     prompt_response_ids = torch.cat(prompt_response_ids, dim=0)
#     action_log_probs = torch.cat(action_log_probs, dim=0)
#     # values = torch.cat(values, dim=0)
#     # returns = torch.cat(returns, dim=0)
#     advantages = torch.cat(advantages, dim=0)
#     attention_mask = torch.cat(attention_mask, dim=0)
#     action_mask = torch.cat(action_mask, dim=0)

#     return BufferItem(
#         prompt_response_ids,
#         action_log_probs,
#         # values,
#         # returns,
#         advantages,
#         attention_mask,
#         action_mask,
#         action_mask.size(1),
#     )

# class PromptDataset(Dataset):
#     def __init__(self, prompts, tokenizer, apply_chat_template=True):
#         self.prompts = prompts
#         self.tokenizer = tokenizer
#         self.final_prompts = []
#         for prompt in self.prompts:
#             if apply_chat_template:
#                 content = [{"role": "user", "content": prompt}]
#                 prompt = self.tokenizer.apply_chat_template(
#                     content, tokenize=False, apply_chat_template=True
#                 )
#             else:
#                 prompt = self.tokenizer.bos_token + prompt
#             self.final_prompts.append(prompt)

#     def __len__(self):
#         return len(self.prompts)

#     def __getitem__(self, idx):
#         return self.final_prompts[idx]


In [7]:
SYSTEM_PROMPT = """
    Use below format to answer questions
    <think>
    your thinking process
    </think>
    <answer>
    your answer
    </answer>
    """

In [ ]:
class GRPOTrainer:
    def __init__(
        self,
        model,
        args,
        train_dataset,
        tokenizer,
        reward_funcs: List[Callable] = None, # here doesn't include loaded reward model from_pretrained
        ref_model=None,
        **kwargs
    ):
        self.model = model.to(model.device)
        self.args = args
        self.ref_model = (
            ref_model.to(model.device).eval()
            if ref_model is not None
            else model.to(model.device).eval()
        )
        self.train_dataset = train_dataset
        self.tokenizer = tokenizer
        self.tokenizer.padding_side = "left"
        self.reward_funcs = reward_funcs
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.args.lr)
        self.max_steps = self.args.max_steps if hasattr(self.args, 'max_steps') else None

    def generate_samples(
        self,
        inputs: List[Dict[str, str]],
    ) -> List[Samples]:
        samples_list = []
        self.model.eval()
        for prompt, answer in zip(inputs["prompt"], inputs["answer"]):
            total_length = self.args.max_generate_length + self.args.max_prompt_length
            input_text = self.tokenizer.apply_chat_template(
                [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt},
                ],
                add_generation_prompt=True,
                tokenize=False,
            )
            inputs = self.tokenizer(
                input_text,
                padding="max_length",
                max_length=self.args.max_prompt_length,
                truncation=True,
                return_tensors="pt",
            )
            input_ids = inputs["input_ids"]
            with torch.no_grad():
                prompt_response_ids = self.model.generate(
                    **inputs.to(self.model.device),
                    max_new_tokens=self.args.max_generate_length,
                    eos_token_id=self.tokenizer.eos_token_id,
                    pad_token_id=self.tokenizer.pad_token_id,
                    num_return_sequences=self.args.num_sample_generations, # generate a group responses
                    do_sample=True,
                )
            if (
                prompt_response_ids.size(1)
                >= total_length
            ):
                prompt_response_ids = prompt_response_ids[
                    :, : total_length
                ]
            else:
                prompt_response_ids = torch.cat(
                    [
                        prompt_response_ids,
                        torch.full(
                            (
                                prompt_response_ids.size(0),
                                total_length
                                - prompt_response_ids.size(1),
                            ),
                            fill_value=self.tokenizer.pad_token_id,
                            device=prompt_response_ids.device,
                        ),
                    ],
                    dim=1,
                )
            attention_mask = (prompt_response_ids.ne(self.tokenizer.pad_token_id)).to(
                dtype=torch.long
            )  # masked since beginning of the prompt
            response_ids = prompt_response_ids[:, input_ids.size(1) :]
            action_mask = (
                response_ids.ne(self.tokenizer.pad_token_id)
                & response_ids.ne(self.tokenizer.eos_token_id)
            ).to(
                dtype=torch.long
            )  # masked since beginning of the answer

            # each samples is a "group" per prompt in GRPO
            samples = Samples(
                prompt_response_ids=prompt_response_ids, # shape: (num_sample_generations, total_length)
                response_ids=response_ids,
                prompt=prompt, # shape: (1)
                answer=answer, # shape: (1)
                attention_mask=attention_mask, # shape: (num_sample_generations, total_length)
                action_mask=action_mask, # shape: (num_sample_generations, max_generate_length)
                num_actions=action_mask.size(1), # shape: (1)
                response_length=action_mask.float().sum(dim=-1), # shape: (num_sample_generations, 1)
            )
            samples_list.append(samples)
        return samples_list

    def generate_experiences(self, samples_list: List[Samples]):
        self.model.eval()
        experiences = []
        for samples in samples_list:
            prompt_response_ids = samples.prompt_response_ids # shape: (num_sample_generations, total_length)
            prompt = samples.prompt # shape: (1)
            answer = samples.answer # shape: (1)
            response_ids = samples.response_ids 
            attention_mask = samples.attention_mask
            action_mask = samples.action_mask
            num_actions = samples.num_actions # shape: (1)
            with torch.no_grad():
                # get output logits
                outputs = self.model(prompt_response_ids, attention_mask=attention_mask)
                log_probs = F.softmax(outputs.logits[:, :-1, :], dim=-1)  # (B, S, V)
                log_probs_labels = log_probs.gather(
                    dim=-1, index=prompt_response_ids[:, 1:].unsqueeze(-1)
                )  # (B, S, 1)
                action_log_probs = log_probs_labels.squeeze(-1)[:, -num_actions:] # truncate for only output part

                responses = self.tokenizer.batch_decode(response_ids, skip_special_tokens=True)
                
                # calculate reward
                # here ignores weights across different reward functions
                prompts = [prompt] * response_ids.size(0)
                answers = [answer] * response_ids.size(0)
                final_reward = torch.zeros(response_ids.size(0), device=self.model.device)
                for reward_func in self.reward_funcs:
                    reward = reward_func(prompts, responses, answers)
                    # Convert list to tensor and accumulate
                    final_reward += torch.tensor(reward, device=self.model.device, dtype=torch.float)
                
                final_reward /= len(self.reward_funcs)
                print(f'prompt: {prompt}, grouped rewards: {final_reward}')

                # noramlize group adv
                final_reward_mean = final_reward.mean()
                final_reward_std = final_reward.std()
                advantages = (final_reward - final_reward_mean) / (final_reward_std + 1e-8)  # (B)

                experiences.append(
                    Experience(
                        prompt_response_ids,
                        action_log_probs.detach(),
                        advantages,
                        attention_mask,
                        action_mask,
                        num_actions
                    )
                )

        return experiences
    
    @staticmethod
    def compute_approx_kl(
        log_probs: torch.Tensor,
        ref_log_probs: torch.Tensor,
        action_mask: Optional[torch.Tensor] = None,
    ):
        # GRPO uses K3 KL divergence
        log_ratio = log_probs.float() - ref_log_probs.float()
        if action_mask is not None:
            log_ratio = log_ratio * action_mask
        kl = log_ratio.exp() - 1 - log_ratio
        return kl
    
    def compute_core_loss(self, new_probs, old_probs, advantages, kl, action_mask=None):
        advantages = advantages.unsqueeze(1)  # shape: (B) -> (B, 1)
        ratio = (new_probs - old_probs).exp()
        loss = (
            -torch.min(
                ratio * advantages, ratio.clamp(1 - self.args.clip_eps, 1 + self.args.clip_eps) * advantages
            )
            + self.args.beta * kl
        )
        if action_mask is None:
            return loss.mean(-1).mean()
        return ((loss * action_mask).sum(-1) / action_mask.sum(-1)).mean()

    
    def compute_loss(self, experiences):
        prompt_response_ids = torch.cat([exp.prompt_response_ids for exp in experiences], dim=0)
        old_action_log_probs = torch.cat([exp.action_log_probs for exp in experiences], dim=0)
        advantages = torch.cat([exp.advantages for exp in experiences], dim=0)
        attention_mask = torch.cat([exp.attention_mask for exp in experiences], dim=0)
        action_mask = torch.cat([exp.action_mask for exp in experiences], dim=0)
        num_actions = experiences[0].num_actions
        
        logits = self.model(prompt_response_ids, attention_mask=attention_mask).logits
        log_probs = F.log_softmax(logits[:, :-1, :], dim=-1)
        log_probs_labels = log_probs.gather(dim=-1, index=prompt_response_ids[:, 1:].unsqueeze(-1))
        action_log_probs = log_probs_labels.squeeze(-1)[:, -num_actions:]

        # get ref's output logits
        ref_output = self.ref_model(prompt_response_ids, attention_mask=attention_mask).logits
        ref_log_probs = F.log_softmax(ref_output[:, :-1, :], dim=-1)
        ref_log_probs_labels = ref_log_probs.gather(
            dim=-1, index=prompt_response_ids[:, 1:].unsqueeze(-1)
        )  # seqs[:, 1:] shape: [batch_size, seq_len-1] – these are the target token IDs (next token at each position) # .unsqueeze(-1) makes that [batch_size, seq_len-1, 1] so it can be used as an index.
        ref_action_log_probs = ref_log_probs_labels.squeeze(-1)[:, -num_actions:] # truncate for only output part

        kl = self.compute_approx_kl(
            action_log_probs, ref_action_log_probs, action_mask=action_mask
        )
        loss = self.compute_core_loss(
            action_log_probs, old_action_log_probs, advantages, kl, action_mask=action_mask
        )
        return loss
    
    def train_step(self, experiences):
        self.model.train()
        loss = self.compute_loss(experiences)
        loss = loss / self.args.gradient_accumulation_steps
        loss.backward()
        print(f"step: {self.step}  policy_loss: {loss.item():.3f}")

        if (self.step + 1) % self.args.gradient_accumulation_steps == 0:
            self.optimizer.step()
            self.optimizer.zero_grad()
            print(f"step {self.step}: optimizer updated!")
        else:
            print(f"step {self.step}: gradients accumulated...")
        
        if (self.step + 1) == self.global_steps:
            self.optimizer.step()
            self.optimizer.zero_grad()

    def train(self):
        self.global_steps = self.args.epoch * len(self.train_dataset) // (self.args.batch_size * self.args.gradient_accumulation_steps)
        self.step = 0
        for _ in range(self.args.epoch):
            dataloader = DataLoader(
                self.train_dataset, batch_size=self.args.batch_size, shuffle=True
            ) # -> Dict[List], e.g. {'prompt': [...], 'answer': [...]}
            for _, batch_input in enumerate(dataloader):
                samples = self.generate_samples(batch_input)
                # print(samples)
                experiences = self.generate_experiences(samples)         
                # self.exps_buffer[idx % self.args.gradient_accumulation_steps] = experiences
                # for step, exp_buffer in enumerate(self.exps_buffer):
                self.train_step(experiences)
                self.step += 1
                if self.max_steps is not None and self.step >= self.max_steps:
                    print(f"Reached max steps of {self.max_steps}, stopping training.")
                    return
                if self.step % self.args.save_steps == 0:
                    self.model.save_pretrained(self.args.output_dir + f'/checkpoint_{self.step}')
                    self.tokenizer.save_pretrained(self.args.output_dir + f'/checkpoint_{self.step}')
                del samples 

In [ ]:
class GRPOArguments:
    output_dir="./grpo/result"
    epoch=1
    lr = 0.000001
    batch_size=2  # 16
    gradient_accumulation_steps=2  # $$\text{Effective Batch Size} = \text{Batch Size per GPU} \times \text{Gradient Accumulation Steps} \times \text{Number of GPUs}$$
    save_steps=100
    num_sample_generations=4  # num_sample genereated per prompt
    max_prompt_length=256
    max_generate_length=512
    reward_weights=None  # only applied if multiple reward funcs are used
    clip_eps=0.2
    beta=0.1
    max_steps = 10

: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args = GRPOArguments()
trainer = GRPOTrainer(model=policy_model,
                        args=args,
                        train_dataset=dataset,
                        tokenizer=tokenizer,
                        reward_funcs = [correctness_reward, digit_reward, hard_format_reward, mark_reward])
trainer.train()


prompt: Bob is building a garden on his land, and he wants it to be fenced in to keep out varmints. The garden is a rectangular plot 225 feet long by 125 feet wide. He also wants to have a small 3-foot wide gate to walk through and a larger 10-foot wide gate to move his gardening equipment through. How much fencing is Bob going to need to fence in his garden, taking into account the two gates?, grouped rewards: tensor([0., 0., 0., 0.])
prompt: Kelly needs school supplies to teach her class for an art project. She has 8 students and they will need 3 pieces of construction paper each. In addition to the construction paper, she needs to buy 6 bottles of glue for everyone to share. After Kelly purchases these supplies, she dropped half of them down a storm drain. Class is about to start, but she manages to rush to the store and buy 5 more pieces of construction paper. How many supplies are left?, grouped rewards: tensor([0., 0., 0., 0.])
